# Tutorial de Ciencia de Datos (CC408) 2024
## Tutorial 11 - PCR, PLS y Modelos no lineales

**Objetivo:**
Que se familiaricen con métodos de regresión basados en reducción de la dimensionalidad (PCR & PLS) y modelos no lineales y semi-paramétricos

Esta tutorial es una adaptación del `Lab` de *Linear Models and Regularization Methods* del libro "Introduction to Statistical Learning with Applications in Python" por Gareth James, Daniela Witten, Trevor Hastie, Robert Tibshirani y Jonathan Taylor. [Acá](https://islp.readthedocs.io/en/latest/labs/Ch06-varselect-lab.html) pueden encontrar más información

### Métodos de Regresión basados en reducción de la dimensionalidad
El objetivo es buscar *transformar* los $p$ predictores y estimar una regresión lineal (por MCO) de $Y$ en los predictores transformados.

#### Regresión por Componentes Principales (PCR, principal component regression)

Para estimar por una regresión por componentes principales (PCR), primero, usamos `PCA` del modulo de [PCA() de Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). 
Luego, usamos la funcion conocida de `LinearRegression()` para estimar el modelo.

En este ejemplo, aplicamos PCR a los datos de [Hitters](https://islp.readthedocs.io/en/latest/datasets/Hitters.html) para predecir `Salary`.

In [1]:
# Importamos los paquetes necesarios
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots

import sklearn.model_selection as skm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from functools import partial

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

##### Preparamos los datos y transformamos variables

In [ ]:
# Importamos los datos datos
Hitters = pd.read_csv('Hitters.csv')
print(Hitters.info())
print('Dimensión de la base:', Hitters.shape, '\n')
#Vemos los missing values en Y
print('\nMissings en variable dependiente:', np.isnan(Hitters['Salary']).sum())

In [ ]:
# Visualizamos la estadistica descriptiva de la base
Hitters.describe().T

In [ ]:
# Eliminamos missings en la variable dependiente
Hitters = Hitters.dropna() 
print('\n Nueva dimensión de la base:', Hitters.shape)

Ahora preparamos nuestras variables com y transformamos las variables string (como *League*) en dummies

In [ ]:
y = Hitters.Salary
# Creamos variables dummies para las variables string
dummies = pd.get_dummies(Hitters[['League', 'Division', 'NewLeague']], drop_first=True)
dummies

# Eliminamos salarios (porque es nuestra y) y las columnas de strings
X_ = Hitters.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)
X.info()

Dividimos la base en observaciones para el entrenamiento y testeo


In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Revisamos cuantas observaciones quedaron para Test y cuantas para Entrenamiento.
print(f'El conjunto de entrenamiento tiene {len(X_train)} observaciones.')
print(f'El conjunto de test tiene {len(X_test)} observaciones.')

In [ ]:
# Iniciamos el Standard Scaler
sc = StandardScaler()

# Estandarizamos las observaciones de entrenamiento
X_train_transformed = pd.DataFrame(sc.fit_transform(X_train), index=X_train.index, columns=X_train.columns)

# Estandarizamos las observaciones de test
X_test_transformed = pd.DataFrame(sc.transform(X_test), index=X_test.index, columns=X_test.columns)

# Estadisticas luego de estandarizar
X_train_transformed.describe().T

##### Regresion por Componentes principales

Utilizamos la función [Pipeline()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) de Sckit learn que permite en una forma clara, separar el paso de *transformar* los predictores y luego *estimar* el modelo de interés. 

In [ ]:
pca = PCA(n_components=2)
linreg = LinearRegression()
pipe = Pipeline([('pca', pca),
                 ('linreg', linreg)])
pipe.fit(X_train_transformed, y_train)
print(f'Los coeficientes de la regresión usando dos componentes principales son:')
print(pipe.named_steps['linreg'].coef_)

Podemos usar cross validation (CV) para buscar el número de componentes a utilizar en la regresión. Para eso, usaremos `skm.GridSearchCV`, donde el parámetro que varia es `n_components`.

In [ ]:
# Usamos 5-fold cross-validation
K = 5
kfold = skm.KFold(K,
                  random_state=0,
                  shuffle=True)

# Definimos un rango de numerop de componentes principales
param_grid = {'pca__n_components': range(1, 20)}


grid = skm.GridSearchCV(pipe,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')
#Estimamos
grid.fit(X_train_transformed, y_train)

Ahora graficamos los resultados

In [ ]:
pcr_fig, ax = subplots(figsize=(6,6))
n_comp = param_grid['pca__n_components']
ax.errorbar(n_comp,
            -grid.cv_results_['mean_test_score'],
            grid.cv_results_['std_test_score'] / np.sqrt(K))
ax.set_ylabel('CV MSE', fontsize=20)
ax.set_xlabel('# componentes principales', fontsize=20)
ax.set_xticks(n_comp[::2])
ax.set_ylim([50000,250000]);

Con el atributo `explained_variance_ratio_` de `PCA` podemos ver el porcentaje de la varianza explicada en los predictores y en la respuesta usando distinto número de componentes principales. 

In [ ]:
pipe.named_steps['pca'].explained_variance_ratio_

Esto nos dice la cantidad de informacion sobre los predictores que es caputara usando $M$ componentes. Esto nos muestra que usar $M=1$ caputa un 38% de la varianza, mientras que $M=2$ captura 22%.

#### Minimos Cuadrados Parciales (Partial Least Squares, PLS)

Minimos Cuadrados Parciales (PLS) se implementa con la funcion [PLSRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.PLSRegression.html)


In [ ]:
pls = PLSRegression(n_components=2, 
                    scale=True)
pls.fit(X_train_transformed, y_train)

Similar al caso de PCR, usamos CV para elegir el numero de componentes

In [ ]:
param_grid = {'n_components':range(1, 20)}

grid = skm.GridSearchCV(pls,
                        param_grid,
                        cv=kfold,
                        scoring='neg_mean_squared_error')

grid.fit(X_train_transformed, y_train)

De la misma forma, en un graficamos el MSE por CV

In [ ]:
pls_fig, ax = subplots(figsize=(6,6))
n_comp = param_grid['n_components']
ax.errorbar(n_comp,
            -grid.cv_results_['mean_test_score'],
            grid.cv_results_['std_test_score'] / np.sqrt(K))
ax.set_ylabel('CV MSE', fontsize=20)
ax.set_xlabel('# componentes', fontsize=20)
ax.set_xticks(n_comp[::2])
ax.set_ylim([50000,250000]);

# Modelos no lineales
En el curso nos enfocamos principalmente en modelos lineales, por ser simples y por sus ventajas en términos de interpretabilidad e inferencia

Sin embargo, el supuesto de linealidad es fuerte y a veces puede llevar a un menor poder predictivo. Ahora vamos a relajar el supuesto de linealidad.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib.pyplot import subplots
#%matplotlib inline

from sklearn import metrics
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score, mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('Wage.csv')
dataset.info()

In [ ]:
# Vamos a usar salario y edad
X = dataset['age']
y = dataset['wage']
print(y)

In [ ]:
fig, ax = subplots(figsize=(8,4))
ax.scatter(X, y, facecolor='gray', alpha=0.5)
ax.set_xlabel('Age')
ax.set_ylabel('Wage')


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Regresión polinómica

In [ ]:
# Reshape para transformar x en un vector columna
X_train_ = X_train.values.reshape((-1, 1)) # Convertir Series a NumPy array y reshape

# Transformación polinomial
model_pol = PolynomialFeatures(include_bias=True, degree=4)
model_pol.fit(X_train_)
X_train_t = model_pol.transform(X_train_)

# Si en PolynomialFeatures ponemos include_bias=False, podemos agregar constante:
#X_train_t = sm.add_constant(X_train_t)

# Especificamos el modelo y ajustamos
model_pol4 = sm.OLS(y_train, X_train_t)
results = model_pol4.fit()

print(results.summary())


Estimamos el MSE de testeo

In [ ]:
X_test_t = model_pol.fit_transform(X_test.values.reshape((-1, 1)))

y_pred = results.predict(X_test_t)

print('ECM:', mean_squared_error(y_test, y_pred))

In [ ]:
# Generamos otras X y sus predicciones para graficar
X_seq = np.linspace(X.min(), X.max()).reshape(-1,1)
X_seq_t = model_pol.fit_transform(X_seq)
X_seq_pred = results.predict(X_seq_t)

fig, ax = subplots(figsize=(8,4))
ax.scatter(X, y, facecolor='gray', alpha=0.5)
ax.plot(X_seq, X_seq_pred, label='Reg. polinómica grado 4', linewidth=3)
ax.set_xlabel('Age')
ax.set_ylabel('Wage')
#ax.legend(title='Poly', fontsize=15)


In [ ]:
results.predict(X_seq_t)

Podemos elegir el grado del polinimio por CV

#### Step function

In [ ]:
# Hacemos a la edad (X) discreta en función de quintiles
cut_X = pd.qcut(X, 4) #qcut con 4 quintiles
cut_X
# Nota pd.cut() permitiría hacer cortes no basados en quintiles

In [ ]:
# y creamos dummies para cada quintil
q_X = pd.get_dummies(cut_X)
q_X

In [ ]:
# Primera columna
q_X.iloc[:, 0]

In [10]:
q_X_train, q_X_test, y_train, y_test = train_test_split(q_X, y, test_size=0.2, random_state=0)

In [ ]:
q_X_train_= sm.add_constant(q_X_train)

# Especificamos el modelo y ajustamos
q_model_pol4 = sm.OLS(y_train, q_X_train_.astype(float))
q_results = q_model_pol4.fit()

print(q_results.summary())


In [ ]:
q_pred =  q_results.predict(sm.add_constant(q_X_test)).values

fig, ax = subplots(figsize=(10,6))
ax.scatter(X, y, facecolor='gray', alpha=0.5)
ax.plot(X_seq, results.predict(X_seq_t), label='Reg. polinómica grado 4', linewidth=4)
ax.scatter(X_test.values.reshape(-1,1), q_pred, facecolor='red', alpha=0.9, label="Step function")
ax.set_xlabel('Age')
ax.set_ylabel('Wage')
ax.legend(title='', fontsize=12)
ax.grid(True)



#### Splines
Idea: usar polinomios y step function

Se determinan puntos de corte (knots) en X y se ajustan distintas regresiones para cada segmento.

In [ ]:
print('Min age:', min(dataset['age']), 'Max age:', max(dataset['age']))

In [ ]:
from scipy.interpolate import BSpline

# Definimos knots y grado del polinomio
knots=[18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78]
degree=3
t = [min(dataset['age'])] + knots + [max(dataset['age'])]
spl = BSpline(t, list(dataset['wage']), degree)

# Grilla para evaluar spline
x_smooth = np.linspace(min(dataset['age']), max(dataset['age']), 300)
y_smooth = spl(x_smooth)

# Plot
fig, ax = subplots(figsize=(8,4))
ax.scatter(X, y, facecolor='gray', alpha=0.5)
ax.plot(X_seq, results.predict(X_seq_t), label='Reg. polinómica grado 4', linewidth=1)
ax.plot(x_smooth, y_smooth, label='Spline', linewidth=2)
ax.set_xlabel('Age')
ax.set_ylabel('Wage')
ax.legend(title='', fontsize=12)


#### Regresión Local

Usamos función [lowess](https://www.statsmodels.org/stable/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html#statsmodels.nonparametric.smoothers_lowess.lowess) de set de funciones no paramétricas de Statsmodels. Esta es una regresión local por Kernels, donde la funcion de Kernels elegida es *tricúbica*.

In [ ]:
import statsmodels as sm

X = dataset['age']
y = dataset['wage']
age_grid = np.linspace(X.min(), X.max(), 100)


lowess = sm.nonparametric.smoothers_lowess.lowess

spans = [0.1, 0.6]
# spans de 0.1 and 0.6. Se consideran 10% o 60% de las observaciones vecinas

# Plot
fig, ax = subplots(figsize=(8,4))
ax.scatter(X, y, facecolor='gray', alpha=0.5)
ax.plot(X_seq, results.predict(X_seq_t), label='Reg. polinómica grado 4', linewidth=2)
for span in spans:
    fitted = lowess(y, X, frac=span, xvals=age_grid)
    ax.plot(age_grid, fitted, label='Reg. local {:.1f}'.format(span), linewidth=2)
ax.set_xlabel('Age')
ax.set_ylabel('Wage')
ax.legend(title='', fontsize=12)



Podemos notar que con span de 0.6 es más suave